# CityShadows: Mapping Tree Clusters
This notebook processes the GeoJSON file generated by `mapping_trees.ipynb`, creating cluster polygons that group trees located in close proximity to one another.
## Libraries

In [ ]:
import geopandas as gpd
import numpy as np
from sklearn.cluster import DBSCAN

## GeoJSON file containing tree data

In [ ]:
trees_dataset_final = gpd.read_file("mapping_heights/trees_dataset.geojson")

## Create clusters

In [ ]:
utm = 32651
trees_utm = trees_dataset.to_crs(epsg=utm).copy()

centroids = trees_utm.geometry.centroid
coords    = np.vstack([centroids.x, centroids.y]).T
db        = DBSCAN(eps=5, min_samples=1).fit(coords) # add eps to make more separated
trees_utm['cluster'] = db.labels_

clustered = (
    trees_utm
    .dissolve(
        by='cluster',
        as_index=False,
        aggfunc={'height':'median'}  
    )
    .rename(columns={'height':'median_height'})
    [['cluster','median_height','geometry']]
)

clustered['geometry'] = clustered.geometry.buffer(2) # buffer to add wrap-around trees

clusters_wgs = clustered.to_crs(epsg=4326)

## Export to GeoJSON

In [ ]:
clusters_wgs.to_file("tree_clusters.geojson", driver="GeoJSON")